In [1]:
import datetime
import pandas as pd
from getpass import getpass
import mysql.connector as mysql
from mysql.connector import connect, Error


##creates connection to SQL server or specific DB if specified
def createConnection(hostName, userName, userPassword, dbName=None):
    connection = None
    try:
        connection = mysql.connect(
            host = hostName,
            user = userName,
          ##for secure signin, use getpass library as follows
          ##passwd = getpass("Enter password:")
            passwd = userPassword,
            database = dbName
        )
        print("Connected!")
    except Error as e:
        print(f"The error '{e}' occurred")
    
    return connection

##creates database with name 'query' in specified server 'connection'
def createDatabase(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute("CREATE DATABASE " + query)
        print("DB Created! " + query)
    except Error as e:
        print(f"the error '{e}' occurred")
        
## executes given SQL query within connect DB provided by createConnection(...)       
def executeQuery(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query committed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
    
def executeReadQuery(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")
        
def createNewTableCSV (connection, csv, table):
    ##connection = createConnection("localhost","root","securePassword", "new_db")
    try:
        if connection.is_connected():
            cursor = connection.cursor(buffered=True)
            cursor.execute("select database();")
            record = cursor.fetchone
            print("You are connected to database: ")
            cursor.execute('DROP TABLE IF EXISTS equities;')
            print('Creating table...')
    ##Pass table here
            cursor.execute(table)
        print("Table created...")
        for i,row in csv.iterrows():
            sql = "INSERT INTO example_database.equities VALUES (%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            connection.commit()
    except Error as e:
        print("Error while connecting to MySQL", e)
            
            
            



In [2]:
connection = createConnection("localhost","root","securePassword")
new_db = "example_database"
createDatabase(connection, new_db)
connection = createConnection("localhost","root","securePassword", "example_database")

Connected!
the error '1007 (HY000): Can't create database 'example_database'; database exists' occurred
Connected!


In [3]:
new_table = """
CREATE TABLE `equities` (
    `stockid` int NOT NULL,
    `symbol` varchar(64) NOT NULL,
    `company_name` varchar(255) NOT NULL,
    `type` varchar(64) NOT NULL,
    `sector` varchar(64) NULL,
    `subsector` varchar(64) NOT NULL
) DEFAULT CHARSET=utf8;
"""
csv = pd.read_csv(r'C:\Users\jackk\Projects\pythonProjects\securities_db\equities.csv', index_col=False, delimiter = ',')

createNewTableCSV(connection, csv, new_table)

You are connected to database: 
Creating table...
Table created...
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Recor

In [4]:
sql = "SELECT * FROM example_database.equities"
cursor = connection.cursor()
cursor.execute(sql)
result = cursor.fetchmany(size=10)
for i in result:
    print(i)

(1, 'A', 'Agilent Technologies Inc', 'Equity', 'Information Technology', '0')
(2, 'AA', 'Alcoa Inc.', 'Equity', 'Materials', '0')
(3, 'AAL', 'American Airls Group Inc', 'Equity', 'Industrials', 'Airlines')
(4, 'AANR', 'Anr Inc Com Cl C New', 'Equity', 'Energy', '0')
(5, 'AANRC', 'Anr Inc Com Cl C New', 'Equity', 'Energy', '0')
(6, 'AAP', 'Advanced Autoparts', 'Equity', 'Consumer Discretionary', '0')
(7, 'AAPL', 'Apple Inc', 'Equity', 'Information Technology', 'Telecommunications Equipment')
(8, 'AAXN', 'Axon Enterprise Inc', 'Equity', 'Industrials', '0')
(9, 'AB', 'Alliancebernstein Hldg LP', 'Equity', 'Financials', '0')
(10, 'ABB', 'ABB Ltd', 'Equity', 'Industrials', '0')


In [5]:
##Update Alcoa Corporation to its proper sector classification
connection = createConnection("localhost","root","securePassword", "example_database")
update_statement = """
Update equities set sector = 'Materials' where symbol = 'AA'
"""

executeQuery(connection, update_statement)

Connected!
Query committed successfully


In [6]:
##Remove bankrupt ticker AANRC from equities table
connection = createConnection("localhost","root","securePassword", "example_database")
delete_statement = """
Delete from equities where stockid = 5
"""

executeQuery(connection, delete_statement)

Connected!
Query committed successfully


In [17]:
##Update stockid to autoincrement going forward
connection = createConnection("localhost","root","securePassword", "example_database")
update_statement = """
ALTER TABLE equities
MODIFY stockid INT NOT NULL AUTO_INCREMENT;
"""

executeQuery(connection, update_statement)

##show successful deletion of bankrupt ticker
sql = "SELECT * FROM example_database.equities"
cursor = connection.cursor()
cursor.execute(sql)
result = cursor.fetchmany(size=20)
for i in result:
    print(i)

Connected!
Query committed successfully
(1, 'A', 'Agilent Technologies Inc', 'Equity', 'Information Technology', '0')
(2, 'AA', 'Alcoa Inc.', 'Equity', 'Materials', '0')
(3, 'AAL', 'American Airls Group Inc', 'Equity', 'Industrials', 'Airlines')
(4, 'AANR', 'Anr Inc Com Cl C New', 'Equity', 'Energy', '0')
(6, 'AAP', 'Advanced Autoparts', 'Equity', 'Consumer Discretionary', '0')
(7, 'AAPL', 'Apple Inc', 'Equity', 'Information Technology', 'Telecommunications Equipment')
(8, 'AAXN', 'Axon Enterprise Inc', 'Equity', 'Industrials', '0')
(9, 'AB', 'Alliancebernstein Hldg LP', 'Equity', 'Financials', '0')
(10, 'ABB', 'ABB Ltd', 'Equity', 'Industrials', '0')
(11, 'ABBV', 'Abbvie Inc', 'Equity', 'Health Care', 'Pharmaceuticals: Major')
(12, 'ABC', 'Amerisourcebergen Corp', 'Equity', 'Health Care', '0')
(13, 'ABI', 'Applera Applied Biosys', 'Equity', 'Health Care', '0')
(14, 'ABN', 'Abn-Amro Holding Nv ADR', 'Equity', 'Financials', '0')
(15, 'ABN+G', 'Abn Amro Cap Fund', 'Equity', '0', '0')
(16